# Task3: MPNN-based Quark/Gluon Jet Classifier



## Data Exploration

In [0]:
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data = np.load('/content/drive/My Drive/QG_jets.npz')
data.files

['X', 'y']

In [4]:
features = data['X']
labels = data['y']
print("features shape:", features.shape)
print("labels shape", labels.shape)
print("features:", features[0])
print("labels", labels[0])

features shape: (100000, 139, 4)
labels shape (100000,)
features: [[ 2.68769142e-01  3.56903171e-01  4.74138734e+00  2.20000000e+01]
 [ 1.60076377e-01 -2.55609533e-01  4.55022910e+00  2.20000000e+01]
 [ 1.14868731e+00 -6.24380156e-02  4.50385377e+00 -2.11000000e+02]
 [ 4.13159146e+00  1.73686350e-01  4.76622410e+00 -3.21000000e+02]
 [ 1.69599701e+00 -2.12177764e-01  4.79687162e+00 -2.11000000e+02]
 [ 2.19372581e+00 -5.24780791e-02  4.57559636e+00  2.20000000e+01]
 [ 1.61909680e+00 -6.76247614e-02  4.64561192e+00  2.20000000e+01]
 [ 6.59214883e+00  4.42691311e-02  4.76597141e+00  2.11000000e+02]
 [ 3.77096258e+00  4.22475280e-02  4.75473207e+00  3.21000000e+02]
 [ 1.34816345e+01 -2.80005472e-02  4.73543183e+00 -2.11000000e+02]
 [ 4.10794493e+00 -2.37648715e-02  4.75891312e+00  2.20000000e+01]
 [ 2.16455176e+01 -2.69973695e-02  4.75997450e+00  2.20000000e+01]
 [ 6.77551168e+00 -2.97549224e-02  4.76127746e+00  2.20000000e+01]
 [ 1.32550803e+01 -3.94389998e-02  4.74948328e+00  2.20000000e+

## Feature Mapping and Clustering

In [5]:
pdgid_class = np.unique(features[:,:,3])
print(pdgid_class)

[-2212. -2112.  -321.  -211.   -13.   -11.     0.    11.    13.    22.
   130.   211.   321.  2112.  2212.]


In [6]:
i = features[0]
np.all(i == 0, axis=-1)

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [7]:
def avg_pt(features):
  return np.array([np.mean(i[~np.all(i==0, axis=-1),0]) for i in features])

def get_pdg_num(feature, pclass):
    return np.count_nonzero(feature[~np.all(feature==0, axis=-1),3] == pclass)

def pdg_num(features):
  return np.array([[get_pdg_num(i, n) for n in (pdgid_class)] for i in features])

average_pt = avg_pt(features)
pdg_number = pdg_num(features)

print(average_pt)
print(pdg_number)

[27.82318056 31.28284093  8.89669563 ... 34.26094921  6.20155106
 13.64612319]
[[0 0 1 ... 1 0 0]
 [0 1 0 ... 0 0 1]
 [2 3 0 ... 0 2 1]
 ...
 [0 1 0 ... 1 0 0]
 [1 2 0 ... 0 2 0]
 [1 2 0 ... 3 2 1]]


In [8]:
print(np.mean(average_pt))
print(np.mean(pdg_number).ravel())

14.555469221796491
[2.88727]


In [9]:
feature_map = np.concatenate((np.reshape(average_pt, (average_pt.size, 1)), pdg_number), axis=1)
print(feature_map.shape)

(100000, 16)


In [10]:
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_map)
_, indices = nbrs.kneighbors(feature_map)
#nbrs.kneighbors_graph(feature_map).toarray() #return adjc matrix

print(indices)

[[    0 17064 70444 11236 28423]
 [    1 53533 28137 21976 90660]
 [    2 99448 28005 32533 74298]
 ...
 [99997  3746  8665  4528 15614]
 [99998 15210 42989 42878 83043]
 [99999  1754 24424 64312 84631]]


## Pytorch Geometric Installation

In [12]:
import torch; print(torch.__version__)

1.4.0


In [13]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install --upgrade --force-reinstall torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/62/20/4d43e141b5bc426ba38274933ef8e76e85c7adea2c321ecf9ebf7421cedf/numpy-1.18.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which 

Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_spline_conv-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-spline-conv 1.2.0
    Uninstalling torch-spline-conv-1.2.0:
      Successfully uninstalled torch-spline-conv-1.2.0


In [14]:
x = np.array([[[i[j], i[j+1], i[j+1], i[j]] for j in range(len(i)-1)] for i in indices]).reshape((800000,2))

x[np.logical_and(np.isin(x[:,0],range(10000)) ,np.isin(x[:,1],range(10000)))].shape

(7886, 2)

## Dataset Definition

In [0]:
import torch
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from tqdm import tqdm


class QGJetsDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(QGJetsDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        pass
    @property
    def processed_file_names(self):
        return ['/content/drive/My Drive/QG_jets_0.dataset']

    def process(self):
        data_list = []
        np_indices = np.array([[[i[j], i[j+1], i[j+1], i[j]] 
                    for j in range(len(i)-1)] for i in indices]).reshape((800000,2))
        #edge_index = torch.tensor(np_indices, dtype=torch.long) #the topK(5) nearest nerbours

        size_data = 10000
        for i in tqdm(range(10)):
          x = torch.tensor(features[size_data*i:size_data*(i+1),:,:].reshape((size_data, -1)), dtype=torch.float) # feature of jets
          y = torch.tensor(labels[size_data*i:size_data*(i+1)], dtype=torch.float) # labels of jet

          sub_edge_index = np_indices[np.logical_and(np.isin(np_indices[:,0],range(size_data)) ,np.isin(np_indices[:,1],range(size_data)))]
          edge_index = torch.tensor(sub_edge_index.T, dtype=torch.long) #the topK(5) nearest nerbours

          data = Data(x=x, edge_index=edge_index, y=y)

          #print(data)

          data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [0]:
def data_loader(train_feature, train_lable, test_feature, test_label, batch_size=256):
    # define a trainset
    trainset = QGJetsDataset(train_feature, train_lable)
    print(trainset)
    # define a trainloader
    trainloader = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
    # define a testset
    testset = QGJetsDataset(test_feature, test_label)
    # define a testloader
    testloader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=False)

    return trainloader, testloader

## Network Definition

In [0]:
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops


class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.fc = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_fc = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]
        x_j = self.fc(x_j)
        x_j = self.act(x_j)       
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]
        new_embedding = torch.cat([aggr_out, x], dim=1)      
        new_embedding = self.update_fc(new_embedding)
        new_embedding = self.update_act(new_embedding)
        return new_embedding

In [0]:
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F


class MPNNet(torch.nn.Module):
    def __init__(self, feature_size):
        super().__init__()

        self.fc0 = torch.nn.Linear(feature_size, 128)

        self.conv1 = SAGEConv(128, 128)
        #self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        #self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        #self.pool3 = TopKPooling(128, ratio=0.8)
        
        self.fc1 = torch.nn.Linear(256, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 1)
        #self.bn1 = torch.nn.BatchNorm1d(128)
        #self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.fc0(x)

        print(x.shape)
        x = x.squeeze(1)        
        print(x.shape)

        x = F.relu(self.conv1(x, edge_index))
        print(x.shape)
        #print(x, edge_index, batch)
        #x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        #x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
        print(x.shape)
        #x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        #x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        #x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        #x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        #x = x1 + x2 + x3
        print(x.shape)

        x = self.fc1(x)
        x = self.act1(x)
        x = self.fc2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.fc3(x)).squeeze(1)

        return x

In [0]:
def train(epoch, model, lossFunction, optimizer, device, trainloader):
    """train model using loss_fn and optimizer. When this function is called, model trains for one epoch.
    Args:
        train_loader: train data
        model: prediction model
        loss_fn: loss function to judge the distance between target and outputs
        optimizer: optimize the loss function
        get_grad: True, False
    output:
        total_loss: loss
        average_grad2: average grad for hidden 2 in this epoch
        average_grad3: average grad for hidden 3 in this epoch
    """
    print('\nEpoch: %d' % epoch)
    model.train()     # enter train mode
    train_loss = 0    # accumulate every batch loss in a epoch
    correct = 0       # count when model' prediction is correct i train set
    total = 0         # total number of prediction in train set
    for batch_idx, data in enumerate(trainloader):
        inputs, targets = data['feature'], data['label']
        inputs, targets = inputs.to(device), targets.to(device) # load data to gpu device
        inputs, targets = Variable(inputs), Variable(targets)
        optimizer.zero_grad()            # clear gradients of all optimized torch.Tensors'
        outputs = model(inputs)          # forward propagation return the value of softmax function
        #print('target', targets)
        targets = targets.squeeze_()
        loss = lossFunction(outputs, targets) #compute loss
        loss.backward()                  # compute gradient of loss over parameters 
        optimizer.step()                 # update parameters with gradient descent 

        train_loss += loss.item()        # accumulate every batch loss in a epoch
        _, predicted = outputs.max(1)    # make prediction according to the outputs
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item() # count how many predictions is correct
        
    print( 'Train loss: %.3f | Train Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
    train_loss = train_loss/(batch_idx+1)
    
    return train_loss
    
    
def test(model, lossFunction, optimizer, device, testloader):
    """
    test model's prediction performance on loader.  
    When thid function is called, model is evaluated.
    Args:
        loader: data for evaluation
        model: prediction model
        loss_fn: loss function to judge the distance between target and outputs
    output:
        total_loss
        accuracy
    """
    model.eval() #enter test mode
    test_loss = 0 # accumulate every batch loss in a epoch
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, data in enumerate(testloader):
            inputs, targets = data['feature'], data['label']
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            targets = targets.squeeze_()
            loss = lossFunction(outputs, targets) #compute loss

            test_loss += loss.item() # accumulate every batch loss in a epoch
            _, predicted = outputs.max(1) # make prediction according to the outputs
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item() # count how many predictions is correct
        # print loss and acc
        print('Test Loss: %.3f  | Test Acc: %.3f%% (%d/%d)'
            % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_loss = test_loss/(batch_idx+1)
        
    return test_loss

In [0]:
def run(model, num_epochs, optimizer, trainloader, testloader, device='cpu', lossFunction=nn.CrossEntropyLoss(), lr=0.01):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    if device == 'cuda':
        model = torch.nn.DataParallel(model)
        torch.backends.cudnn.benchmark=True

    train_loss_list = []
    test_loss_list = []

    for epoch in range(num_epochs):
        train_loss_list.append(train(epoch, model, lossFunction, optimizer, device, trainloader))
        test_loss_list.append(test(model, lossFunction, optimizer, device, testloader))

    return train_loss_list, test_loss_list

## Testing

In [72]:
dataset = QGJetsDataset('/content/drive/My Drive')
dataset.shuffle()
print(dataset)
train_loader = DataLoader(dataset[int(0.9*len(dataset)):], batch_size=256, shuffle=True)
test_loader = DataLoader(dataset[:int(0.9*len(dataset))], batch_size=256, shuffle=True)
for data in train_loader:
  print(data)
  print(data.num_graphs)

QGJetsDataset(10)
Batch(batch=[10000], edge_index=[2, 7886], x=[10000, 556], y=[10000])
1


In [53]:
np_indices = np.array([[[i[j], i[j+1], i[j+1], i[j]] 
  for j in range(len(i)-1)] for i in indices]).reshape((800000,2))
edge_index = torch.tensor(np_indices.T, dtype=torch.long) #the topK(5) nearest nerbours
x = torch.tensor(features.reshape((100000, -1)), dtype=torch.float) # feature of jets
y = torch.tensor(labels, dtype=torch.float) # labels of jet

data = Data(x=x, edge_index=edge_index, y=y)

print(data)

print(data.num_nodes, data.num_edges, data.num_node_features)
print(data.contains_isolated_nodes(), data.contains_self_loops(), data.is_directed())

Data(edge_index=[2, 800000], x=[100000, 556], y=[100000])
100000 800000 556
False False False


In [0]:
from torch_geometric.nn import GCNConv
from torch.nn import BatchNorm1d

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc0 = torch.nn.Linear(556, 128)
        self.bn0 = BatchNorm1d(128)
        self.conv1 = SAGEConv(128, 128)
        self.bn1 = BatchNorm1d(128)
        self.conv2 = SAGEConv(128, 128)
        self.bn2 = BatchNorm1d(128)
        self.conv3 = SAGEConv(128, 128)
        self.bn3 = BatchNorm1d(128)
        self.fc1 = torch.nn.Linear(128, 64)
        self.fc2 = torch.nn.Linear(64, 32)
        self.fc3 = torch.nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.fc0(x)
        x = self.bn0(x)

        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)

        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = F.relu(x)

        x = self.fc3(self.fc2(self.fc1(x)))

        #print('last layer:', x.view(-1))

        out = F.softmax(x, dim=1)

        return out

In [137]:
num_epochs = 10
batch_size = 256
lr = 0.01
device='cpu'
crit = nn.BCELoss()
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
print(model)

Net(
  (fc0): Linear(in_features=556, out_features=128, bias=True)
  (bn0): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): SAGEConv(
    (fc): Linear(in_features=128, out_features=128, bias=True)
    (act): ReLU()
    (update_fc): Linear(in_features=256, out_features=128, bias=False)
    (update_act): ReLU()
  )
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): SAGEConv(
    (fc): Linear(in_features=128, out_features=128, bias=True)
    (act): ReLU()
    (update_fc): Linear(in_features=256, out_features=128, bias=False)
    (update_act): ReLU()
  )
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): SAGEConv(
    (fc): Linear(in_features=128, out_features=128, bias=True)
    (act): ReLU()
    (update_fc): Linear(in_features=256, out_features=128, bias=False)
    (update_act): ReLU()
  )
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine

In [0]:
def train(data):
    model.train()

    loss_all = 0
    #for data in train_loader:

    data = data.to(device)
    optimizer.zero_grad()
    output = model(data)

    label = data.y.view(-1, 1)

    loss = crit(output, label)
    loss.backward()
    optimizer.step()

    return loss

In [0]:
from sklearn.metrics import roc_auc_score
def evaluate(data):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        #for data in loader:

        data = data.to(device)
        pred = model(data).detach().cpu().numpy()

        label = data.y.detach().cpu().numpy()
        #print(pred[:20])
        #print(label[:20])

        predictions.append(pred)
        labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [0]:
for epoch in range(1, 10):
    loss = train(data)
    print('Epoch: {:03d}, Loss: {:.5f}}'.
          format(epoch, loss))

In [139]:
for epoch in range(1, 10):
    loss = train(data)
    train_acc = evaluate(data)
    val_acc = evaluate(data)    
    test_acc = evaluate(data)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

Epoch: 001, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 002, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 003, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 004, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 005, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 006, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 007, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 008, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
Epoch: 009, Loss: 13.91266, Train Auc: 0.50000, Val Auc: 0.50000, Test Auc: 0.50000
